<a href="https://colab.research.google.com/github/Ali-mohammadi-design/Prompt_Engineering_and_Machine_Learning/blob/main/Prompt_engineering_AWS_Bedrock_API_Adjusting_temperature_%26_TopP__Repeatable_Results_Amazon_Titan_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Note: In this training we want to set the tempertaure and Top P so that the LLM give us the same response whenever we send it a request. The first part of this file is about setting AWS_bedrock API and Models and the second part is about Top P and Temperature. To better Understand the concept of Top p and Temperature and other general parameters in the LLMS we can refer to the following article written in linked in: https://www.linkedin.com/pulse/science-control-how-temperature-topp-topk-shape-large-puente-viejo-u88yf/
**
> Indented block



In [78]:
! pip install boto3

In [79]:
import boto3
s3=boto3.resource('s3')

In [80]:
s3.buckets.all()

s3.bucketsCollection(s3.ServiceResource(), s3.Bucket)

In [81]:
import os
aws_access_key_id=os.environ['aws_access_key_id']
aws_secret_access_key=os.environ['aws_secret_access_key']

In [82]:
s3 = boto3.resource('s3',
         aws_access_key_id=aws_access_key_id,
         aws_secret_access_key= aws_secret_access_key)

In [83]:
for bucket in s3.buckets.all():
  print(bucket.name)

bounding-box-labling-job
image-labeling-semantic-1
my-first-buckettest
my-lableing-job
my-second-labling-job
my-third-labling-job
myfirstcanvus
sagemaker-studio-019966197535-0mhpsupvuu6a
sagemaker-studio-019966197535-ajdlxlmu49c
sagemaker-us-east-1-019966197535
text-labeling-1


In [84]:
bedrock=boto3.client(aws_access_key_id=aws_access_key_id,aws_secret_access_key= aws_secret_access_key, service_name='bedrock', region_name='us-east-1')

Note: with the following function you can see all the available models. To use a model you need Model ID

In [157]:
#bedrock.list_foundation_models()

You can get the information of an specific model with its model ID.

In [86]:
bedrock.get_foundation_model(modelIdentifier='amazon.titan-text-express-v1')

{'ResponseMetadata': {'RequestId': 'b06f69a7-9a8a-4802-885a-eabc3397d71d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 27 Feb 2024 23:00:05 GMT',
   'content-type': 'application/json',
   'content-length': '402',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b06f69a7-9a8a-4802-885a-eabc3397d71d'},
  'RetryAttempts': 0},
 'modelDetails': {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-text-express-v1',
  'modelId': 'amazon.titan-text-express-v1',
  'modelName': 'Titan Text G1 - Express',
  'providerName': 'Amazon',
  'inputModalities': ['TEXT'],
  'outputModalities': ['TEXT'],
  'responseStreamingSupported': True,
  'customizationsSupported': [],
  'inferenceTypesSupported': ['ON_DEMAND'],
  'modelLifecycle': {'status': 'ACTIVE'}}}

In [116]:
question='what is the largest city in Canada?'

Note: to be realted to the LLMs we have to activate bedrock with bedrock run time

In [117]:
bedrock_runtime= boto3.client(aws_access_key_id=aws_access_key_id,aws_secret_access_key= aws_secret_access_key, service_name='bedrock-runtime', region_name='us-east-1')

Note: In berock we would tlak with LLMS in json format

In [138]:
import json

In [139]:
body=json.dumps({'inputText':question })

Note: json.dumps would change the json to string

In [140]:
body

'{"inputText": "what is the largest city in Canada?"}'

In [141]:
type(body)

str

Note: To use the models you have to first activate accessability of the models on AWS website, manage model access section.

In [142]:
response=bedrock_runtime.invoke_model(body=body, modelId='amazon.titan-text-express-v1')

In [143]:
response

{'ResponseMetadata': {'RequestId': 'a9dc1a87-7717-4684-9be6-089b7a6a0d45',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 27 Feb 2024 23:14:24 GMT',
   'content-type': 'application/json',
   'content-length': '138',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a9dc1a87-7717-4684-9be6-089b7a6a0d45',
   'x-amzn-bedrock-invocation-latency': '613',
   'x-amzn-bedrock-output-token-count': '9',
   'x-amzn-bedrock-input-token-count': '8'},
  'RetryAttempts': 0},
 'contentType': 'application/json',
 'body': <botocore.response.StreamingBody at 0x7fdc83e3ffa0>}

Note: The response is on body. Thus wehave to read the body with the following code

In [144]:
answer=response.get('body').read()

In [145]:
print(answer)

b'{"inputTextTokenCount":8,"results":[{"tokenCount":9,"outputText":"\\nToronto is the largest city in Canada.","completionReason":"FINISH"}]}'


In [148]:
response_body=json.loads(answer)

In [153]:
print(response_body['results'][0]['outputText'])


Toronto is the largest city in Canada.


In this section we would show that by adjusting the parameters of models we can affect on the output of them. For instance by just setting the temperture and P factor we can provide a situation that recieve the same answer every time we send a request. Even if the request is about writing a creative matrerial

In [154]:
request='Tell me a story about a magic forest'

Note: As is clear in the following code, since we put the temperature on 0 we would set the randomeness on its minimum. Setting top p on 0.01 (it cannot be set on 0) would give us very small set of words to be selected. Thus the output should be same whenever we would send an input to the LLM.

In [156]:
body2=json.dumps({'inputText':request, 'textGenerationConfig':{'temperature':0, 'topP':0.01, 'maxTokenCount':512} })

In [158]:
bedrock_runtime= boto3.client(aws_access_key_id=aws_access_key_id,aws_secret_access_key= aws_secret_access_key, service_name='bedrock-runtime', region_name='us-east-1')

In [159]:
response2=bedrock_runtime.invoke_model(body=body2, modelId='amazon.titan-text-express-v1')

In [160]:
response2

{'ResponseMetadata': {'RequestId': '3a4e14d8-fe74-4433-ac38-a16c1568b2f7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 27 Feb 2024 23:25:24 GMT',
   'content-type': 'application/json',
   'content-length': '2435',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3a4e14d8-fe74-4433-ac38-a16c1568b2f7',
   'x-amzn-bedrock-invocation-latency': '16855',
   'x-amzn-bedrock-output-token-count': '512',
   'x-amzn-bedrock-input-token-count': '8'},
  'RetryAttempts': 0},
 'contentType': 'application/json',
 'body': <botocore.response.StreamingBody at 0x7fdc83e3cb20>}

In [162]:
answer2=response2.get('body').read()

In [164]:
response_body2=json.loads(answer2)

In [166]:
#response_body2

In [168]:
print(response_body2['results'][0]['outputText'])


Once upon a time, in a faraway land, there was a magical forest. The forest was said to be home to all sorts of magical creatures, from fairies to unicorns to dragons. One day, a young girl named Lily stumbled upon the forest while on a walk with her parents. She was amazed by the beauty of the forest and the creatures that lived there.

Lily decided to explore the forest and see what she could find. As she walked deeper into the forest, she saw a group of fairies dancing in a clearing. They were so beautiful that Lily couldn't help but stop and watch them.

The fairies noticed her and invited her to join them in their dance. Lily was hesitant at first, but the fairies were so friendly and welcoming that she soon felt at ease. She danced with the fairies for hours, and before she knew it, the sun had set and it was time to go home.

As Lily walked back to her parents' house, she couldn't stop thinking about the magical forest. She knew that she would never forget her adventure and tha

Now we send the equal request again to the LLM to see the difference of out put. Just compare the previous out put with the new one!

In [170]:
bedrock_runtime= boto3.client(aws_access_key_id=aws_access_key_id,aws_secret_access_key= aws_secret_access_key, service_name='bedrock-runtime', region_name='us-east-1')


In [171]:
response3=bedrock_runtime.invoke_model(body=body2, modelId='amazon.titan-text-express-v1')
answer3=response3.get('body').read()
response_body3=json.loads(answer3)
print(response_body3['results'][0]['outputText'])


Once upon a time, in a faraway land, there was a magical forest. The forest was said to be home to all sorts of magical creatures, from fairies to unicorns to dragons. One day, a young girl named Lily stumbled upon the forest while on a walk with her parents. She was amazed by the beauty of the trees and the colors of the flowers. As she walked deeper into the forest, she saw a group of fairies dancing in a clearing. They were so beautiful that Lily couldn't help but stop and watch. The fairies invited her to join them, and she danced and laughed with them for hours.

As the sun began to set, Lily realized that she had to get back home. But she promised to come back to the magical forest again soon. And she did. Every day after that, Lily would go to the forest and spend hours exploring its wonders. She met all sorts of magical creatures, and she learned about their powers and their secrets.

One day, while Lily was wandering through the forest, she came across a small, injured unicor